In [1]:
import pandas as pd
import numpy as np
import random
import csv
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import RadiusNeighborsClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.svm import LinearSVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegressionCV
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
#from sklearn.mixture import DPGMM
from tabulate import tabulate

In [2]:
import sys
sys.path.append('../automated_model_creation') #We need to access the function file
from functions import *

In [3]:
df = pd.read_csv("../data/kaggle_data/ufc-master.csv")
df_upcoming = pd.read_csv('../data/kaggle_data/upcoming-event.csv')
len(df)



4793

In [4]:
num_upcoming_fights = len(df_upcoming)

In [5]:
df = df_upcoming.append(df)
len(df)

4805

In [6]:
df['date'] = pd.to_datetime(df['date'])

In [7]:
def return_finish_type(winner, finish):
    #print(winner, finish)
    #Why overcomplicate things?  We can just use a few if statements
    if winner == 'Red':
        #print("HI")
        if finish in ['U-DEC', 'S-DEC', 'M-DEC']:
            return ('Red - DEC')
        if finish in ['SUB']:
            return('Red - SUB')
        if finish in ['KO/TKO', 'DQ']:
            return('Red - KO/TKO')
    if winner == 'Blue':
        if finish in ['U-DEC', 'S-DEC', 'M-DEC']:
            return ('Blue - DEC')
        if finish in ['SUB']:
            return('Blue - SUB')
        if finish in ['KO/TKO', 'DQ']:
            return('Blue - KO/TKO')
        
    #Test for NaN
    if finish != finish:
        return('')
    
    if finish == 'Overturned':
        return('')
    
    
    return ('error')

In [8]:
#This calls for the power of lambda!
df['finish_type'] = df.apply(lambda x: return_finish_type(x['Winner'], x['finish']), axis=1)

In [9]:
finish_list = ['Red - DEC', 'Red - SUB', 'Red - KO/TKO', 'Blue - DEC', 'Blue - SUB', 'Blue - KO/TKO']

#Let's put all the labels in a dataframe
df['label'] = ''
#If the winner is not Red or Blue we can remove it.

for f in range(len(finish_list)):
    mask = df['finish_type'] == finish_list[f]
    df['label'][mask] = f
    
#df["Winner"] = df["Winner"].astype('category')
#df = df[(df['Winner'] != 'Blue') | (df['Winner'] == 'Red') ]


#Make sure lable is numeric
df['label'] = pd.to_numeric(df['label'], errors='coerce')

<ipython-input-9-af8f7542de83>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][mask] = f


In [10]:
df.rename(columns={'r_dec_odds': 'Red - DEC', 'r_sub_odds': 'Red - SUB', 'r_ko_odds': 'Red - KO/TKO',
                'b_dec_odds': 'Blue - DEC', 'b_sub_odds': 'Blue - SUB', 'b_ko_odds': 'Blue - KO/TKO'}, inplace=True)

In [11]:
df_label = df['label']
df_odds = df[finish_list]

In [12]:
df_train = df[num_upcoming_fights:]
odds_train = df_odds[num_upcoming_fights:]
label_train = df_label[num_upcoming_fights:]

df_test = df[:num_upcoming_fights]
odds_test = df_odds[:num_upcoming_fights]
label_test = df_label[:num_upcoming_fights]

print(len(df_test))
print(len(odds_test))
print(len(label_test))

print(len(df_train))
print(len(odds_train))
print(len(label_train))

12
12
12
4793
4793
4793


In [13]:
weightclass_list = ['B_match_weightclass_rank', 'R_match_weightclass_rank', "R_Women's Flyweight_rank", "R_Women's Featherweight_rank", "R_Women's Strawweight_rank", "R_Women's Bantamweight_rank", 'R_Heavyweight_rank', 'R_Light Heavyweight_rank', 'R_Middleweight_rank', 'R_Welterweight_rank', 'R_Lightweight_rank', 'R_Featherweight_rank', 'R_Bantamweight_rank', 'R_Flyweight_rank', 'R_Pound-for-Pound_rank', "B_Women's Flyweight_rank", "B_Women's Featherweight_rank", "B_Women's Strawweight_rank", "B_Women's Bantamweight_rank", 'B_Heavyweight_rank', 'B_Light Heavyweight_rank', 'B_Middleweight_rank', 'B_Welterweight_rank', 'B_Lightweight_rank', 'B_Featherweight_rank', 'B_Bantamweight_rank', 'B_Flyweight_rank', 'B_Pound-for-Pound_rank']
df_train[weightclass_list] = df_train[weightclass_list].fillna(17)
df_test[weightclass_list] = df_test[weightclass_list].fillna(17)

C:\Users\matth\anaconda3\lib\site-packages\pandas\core\frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [14]:
display(df_train)
display(df_test)

,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,finish_round_time,total_fight_time_secs,Red - DEC,Blue - DEC,Red - SUB,Blue - SUB,Red - KO/TKO,Blue - KO/TKO,finish_type,label
0,Islam Makhachev,Thiago Moises,-900,600,11.111111,600.000000,2021-07-17,"Las Vegas, Nevada, USA",USA,Red,...,2:38,1058.0,-105.0,1000.0,275.0,1800.0,330.0,1600.0,Red - SUB,1.0
1,Marion Reneau,Miesha Tate,155,-180,155.000000,55.555556,2021-07-17,"Las Vegas, Nevada, USA",USA,Blue,...,1:53,713.0,240.0,110.0,700.0,500.0,1100.0,1200.0,Blue - KO/TKO,5.0
2,Jeremy Stephens,Mateusz Gamrot,220,-275,220.000000,36.363636,2021-07-17,"Las Vegas, Nevada, USA",USA,Blue,...,1:05,65.0,500.0,150.0,1600.0,1100.0,450.0,180.0,Blue - SUB,4.0
3,Rodolfo Vieira,Dustin Stoltzfus,-210,175,47.619048,175.000000,2021-07-17,"Las Vegas, Nevada, USA",USA,Red,...,1:54,714.0,400.0,400.0,-110.0,1400.0,800.0,500.0,Red - SUB,1.0
4,Gabriel Benitez,Billy Quarantillo,-150,130,66.666667,130.000000,2021-07-17,"Las Vegas, Nevada, USA",USA,Blue,...,3:40,820.0,150.0,350.0,600.0,850.0,350.0,500.0,Blue - KO/TKO,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4788,Duane Ludwig,Darren Elkins,-155,135,64.516129,135.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Blue,...,0:44,44.0,NaN,NaN,NaN,NaN,NaN,NaN,Blue - KO/TKO,5.0
4789,John Howard,Daniel Roberts,-210,175,47.619048,175.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,2:01,121.0,NaN,NaN,NaN,NaN,NaN,NaN,Red - KO/TKO,2.0
4790,Brendan Schaub,Chase Gormley,-260,220,38.461538,220.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,0:47,47.0,NaN,NaN,NaN,NaN,NaN,NaN,Red - KO/TKO,2.0
4791,Mike Pierce,Julio Paulino,-420,335,23.809524,335.000000,2010-03-21,"Broomfield, Colorado, USA",USA,Red,...,5:00,900.0,NaN,NaN,NaN,NaN,NaN,NaN,Red - DEC,0.0


,R_fighter,B_fighter,R_odds,B_odds,R_ev,B_ev,date,location,country,Winner,...,finish_round_time,total_fight_time_secs,Red - DEC,Blue - DEC,Red - SUB,Blue - SUB,Red - KO/TKO,Blue - KO/TKO,finish_type,label
0,Cory Sandhagen,TJ Dillashaw,-195,165,51.282051,165.000000,2021-07-24,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,450.0,600.0,600.0,1000.0,120.0,350.0,,NaN
1,Aspen Ladd,Macy Chiasson,-200,170,50.000000,170.000000,2021-07-24,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,150.0,300.0,700.0,1400.0,275.0,700.0,,NaN
2,Kyler Phillips,Raulian Paiva,-260,210,38.461538,210.000000,2021-07-24,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,-105.0,450.0,800.0,1200.0,350.0,800.0,,NaN
3,Darren Elkins,Darrick Minner,130,-155,130.000000,64.516129,2021-07-24,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,275.0,240.0,500.0,250.0,900.0,700.0,,NaN
4,Miranda Maverick,Maycee Barber,-150,130,66.666667,130.000000,2021-07-24,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,120.0,275.0,600.0,1200.0,1000.0,400.0,,NaN
5,Mickey Gall,Jordan Williams,150,-170,150.000000,58.823529,2021-07-24,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,400.0,225.0,400.0,1100.0,1100.0,175.0,,NaN
6,Punahele Soriano,Brendan Allen,-110,-110,90.909091,90.909091,2021-07-24,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,555.0,450.0,NaN,225.0,NaN,700.0,,NaN
7,Nassourdine Imavov,Ian Heinisch,130,-150,130.000000,66.666667,2021-07-24,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,250.0,140.0,800.0,1000.0,700.0,450.0,,NaN
8,Adrian Yanez,Randy Costa,-220,180,45.454545,180.000000,2021-07-24,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,350.0,500.0,1000.0,1600.0,-110.0,450.0,,NaN
9,Julio Arce,Andre Ewell,-200,170,50.000000,170.000000,2021-07-24,"Las Vegas, Nevada, USA",USA,NaN,...,NaN,NaN,110.0,300.0,450.0,1400.0,600.0,700.0,,NaN


In [15]:
with open('../data/production_models_mov.csv', newline='') as f:
    reader = csv.reader(f)
    models = list(reader)
    
#print(len(models))

#set the model choice automatically:
model_choice = 0
best_score = models[4][0]
for n in range(len(models[0])):
    print(n)
    if (float(models[4][n]) > float((best_score))):
        print("YES")
        best_score = models[4][n]
        model_choice = n

print(f"The model choice is {model_choice}")
print(models)

###REVERT THIS
#model_choice = 0

###


model = eval(models[1][model_choice])
features = eval(models[2][model_choice])
ev = eval(models[3][model_choice])
print()
print(model, features, ev)

0
The model choice is 0
[['model_2021_07_15'], ['LinearDiscriminantAnalysis()'], ['["R_Women\'s Featherweight_rank", \'B_Lightweight_rank\', \'R_Featherweight_rank\', \'R_Light Heavyweight_rank\', \'total_title_bout_dif\', \'R_losses\', \'B_win_by_Submission\', \'Blue - DEC\', \'B_draw\', \'R_Middleweight_rank\', \'B_avg_SIG_STR_pct\', "B_Women\'s Strawweight_rank", \'age_dif\', \'B_Middleweight_rank\', \'avg_sub_att_dif\', \'avg_td_dif\', \'B_avg_SUB_ATT\', "R_Women\'s Flyweight_rank", \'R_Pound-for-Pound_rank\', \'R_Reach_cms\', \'R_wins\', \'B_ev\', \'R_Weight_lbs\', \'sub_dif\', \'R_ev\', \'B_avg_TD_landed\']'], ['0'], ['7360.0']]

LinearDiscriminantAnalysis() ["R_Women's Featherweight_rank", 'B_Lightweight_rank', 'R_Featherweight_rank', 'R_Light Heavyweight_rank', 'total_title_bout_dif', 'R_losses', 'B_win_by_Submission', 'Blue - DEC', 'B_draw', 'R_Middleweight_rank', 'B_avg_SIG_STR_pct', "B_Women's Strawweight_rank", 'age_dif', 'B_Middleweight_rank', 'avg_sub_att_dif', 'avg_td_di

In [16]:
def display_bets(df, min_ev, finish_list, save_to_html=False):
    bet_list = []
    print("min_ev = " + str(min_ev))
    for index, row in df.iterrows():
        print()
        print()
        print(f"{row['t1_name']} (RED) vs. {row['t2_name']} (BLUE)")
        for f in finish_list:
            ev = get_bet_ev(row[f], row[f+'_prob'])
            if ev > min_ev:
                print(f"BET: {f}. ODDS: {row[f]}. PROB: {row[f+'_prob']}.  EV: {ev}")
                bet_list.append([row['t1_name'], row['t2_name'], f, row[f]])
                
    #print(bet_list)
    export_bets = pd.DataFrame(bet_list, columns=['Red', 'Blue', 'Bet', 'Odds'])
    if save_to_html:
        export_bets.to_csv('bets.csv', index=False)

In [17]:
def get_bets(input_model, input_features, input_ev, train_df, train_labels, train_odds, test_df, test_labels,
             test_odds, finish_list, verbose=True, save_to_html=False):
    fighters_test = df[['R_fighter', 'B_fighter']]
    
    df_train = train_df[input_features].copy()
    df_test = test_df[input_features].copy()
    df_train = df_train.dropna()
    df_test = df_test.dropna()
        
    df_train = pd.get_dummies(df_train)
    df_test = pd.get_dummies(df_test)
    df_train, df_test = df_train.align(df_test, join='left', axis=1)    #Ensures both sets are dummified the same
    df_test = df_test.fillna(0)

    labels_train = train_labels[train_labels.index.isin(df_train.index)]
    odds_train = train_odds[train_odds.index.isin(df_train.index)] 
    #print(labels_train)
    print(len(df_train))
    labels_test = test_labels[test_labels.index.isin(df_test.index)]
    odds_test = test_odds[test_odds.index.isin(df_test.index)] 
    fighters_test = fighters_test[fighters_test.index.isin(df_test.index)]
    
    labels_train = labels_train.dropna()
    odds_train = odds_train[odds_train.index.isin(labels_train.index)]     
    df_train = df_train[df_train.index.isin(labels_train.index)] 

    
    
    #input_model.fit_transform(df_train, labels_train)
    #probs = input_model.predict_proba(df_test)
    
    
    
    scaler = StandardScaler()
    scaled_train = scaler.fit_transform(df_train)

    input_model.fit(scaled_train, labels_train)

    #print(scaled_train)
    
    scaled_test = scaler.transform(df_test)

    probs = input_model.predict_proba(scaled_test)    

    
    
    
    
    
    
    
    
    
    prob_label_list = []
    for f in finish_list:
        prob_label_list.append(f + '_prob')
    odds_test = np.array(odds_test)  
    fighters_test = np.array(fighters_test)
    #print(probs)
    #print()
    #print()
    prepped_test = list(zip(fighters_test[:, -2], fighters_test[:, -1], odds_test[:, 0], odds_test[:, 1],
                            odds_test[:, 2], odds_test[:, 3],odds_test[:, 4], odds_test[:, 5],
                            probs[:, 0], probs[:, 1],probs[:, 2], probs[:, 3],probs[:, 4], probs[:, 5]))
    ev_prepped_df = pd.DataFrame(prepped_test, columns=['t1_name', 't2_name'] + finish_list + prob_label_list)

    display_bets(ev_prepped_df, input_ev, finish_list, save_to_html=save_to_html)
    
    return(ev_prepped_df)

In [18]:
output_df = get_bets(model, features, ev, df_train, label_train, odds_train, df_test, label_test,
                         odds_test, finish_list, verbose = True, save_to_html=True)

3365
min_ev = 0


Cory Sandhagen (RED) vs. TJ Dillashaw (BLUE)
BET: Red - DEC. ODDS: 450.0. PROB: 0.22317163626385783.  EV: 22.7443999451218


Aspen Ladd (RED) vs. Macy Chiasson (BLUE)
BET: Red - DEC. ODDS: 150.0. PROB: 0.42267943085291954.  EV: 5.669857713229888


Kyler Phillips (RED) vs. Raulian Paiva (BLUE)


Darren Elkins (RED) vs. Darrick Minner (BLUE)
BET: Blue - SUB. ODDS: 250.0. PROB: 0.4221637853033859.  EV: 47.75732485618507


Miranda Maverick (RED) vs. Maycee Barber (BLUE)
BET: Red - DEC. ODDS: 120.0. PROB: 0.49020970792616486.  EV: 7.846135743756278
BET: Red - KO/TKO. ODDS: 1000.0. PROB: 0.09110988165870446.  EV: 0.22086982457490478


Mickey Gall (RED) vs. Jordan Williams (BLUE)
BET: Red - SUB. ODDS: 400.0. PROB: 0.3052848835338068.  EV: 52.64244176690339
BET: Red - KO/TKO. ODDS: 1100.0. PROB: 0.09371720693345394.  EV: 12.460648320144728


Punahele Soriano (RED) vs. Brendan Allen (BLUE)
BET: Red - DEC. ODDS: 555.0. PROB: 0.3094583013333058.  EV: 102.69518737331532


Nassour

In [19]:
output_df

,t1_name,t2_name,Red - DEC,Red - SUB,Red - KO/TKO,Blue - DEC,Blue - SUB,Blue - KO/TKO,Red - DEC_prob,Red - SUB_prob,Red - KO/TKO_prob,Blue - DEC_prob,Blue - SUB_prob,Blue - KO/TKO_prob
0,Cory Sandhagen,TJ Dillashaw,450.0,600.0,120.0,600.0,1000.0,350.0,0.223172,0.075845,0.411871,0.110798,0.059153,0.119162
1,Aspen Ladd,Macy Chiasson,150.0,700.0,275.0,300.0,1400.0,700.0,0.422679,0.114853,0.121510,0.200198,0.025593,0.115166
2,Kyler Phillips,Raulian Paiva,-105.0,800.0,350.0,450.0,1200.0,800.0,0.501215,0.100806,0.139857,0.140114,0.020188,0.097819
3,Darren Elkins,Darrick Minner,275.0,500.0,900.0,240.0,250.0,700.0,0.215185,0.072782,0.098593,0.122071,0.422164,0.069205
4,Miranda Maverick,Maycee Barber,120.0,600.0,1000.0,275.0,1200.0,400.0,0.490210,0.059165,0.091110,0.258121,0.022148,0.079246
5,Mickey Gall,Jordan Williams,400.0,400.0,1100.0,225.0,1100.0,175.0,0.155383,0.305285,0.093717,0.228700,0.025207,0.191708
6,Punahele Soriano,Brendan Allen,555.0,NaN,NaN,450.0,225.0,700.0,0.309458,0.039920,0.188341,0.142780,0.205734,0.113767
7,Nassourdine Imavov,Ian Heinisch,250.0,800.0,700.0,140.0,1000.0,450.0,0.179606,0.181171,0.138963,0.255845,0.043613,0.200802
8,Adrian Yanez,Randy Costa,350.0,1000.0,-110.0,500.0,1600.0,450.0,0.402742,0.076357,0.184275,0.181377,0.026495,0.128753
9,Julio Arce,Andre Ewell,110.0,450.0,600.0,300.0,1400.0,700.0,0.362937,0.118086,0.146423,0.231981,0.022388,0.118185
